<a href="https://colab.research.google.com/github/vaish2814/Machine_Learning/blob/main/final_food_delivery_dataset_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import sqlite3


In [2]:
orders = pd.read_csv("/content/orders.csv")
orders.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [4]:
users = pd.read_json("/content/users.json")
users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [5]:
conn = sqlite3.connect("restaurants.db")
cursor = conn.cursor()

with open("/content/restaurants.sql", "r") as file:
    sql_script = file.read()

cursor.executescript(sql_script)
conn.commit()


In [6]:
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [7]:
merged_df = orders.merge(
    users,
    on="user_id",
    how="left"
)


In [8]:
final_df = merged_df.merge(
    restaurants,
    on="restaurant_id",
    how="left"
)


In [9]:
final_df.info()
final_df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   order_id           10000 non-null  int64  
 1   user_id            10000 non-null  int64  
 2   restaurant_id      10000 non-null  int64  
 3   order_date         10000 non-null  object 
 4   total_amount       10000 non-null  float64
 5   restaurant_name_x  10000 non-null  object 
 6   name               10000 non-null  object 
 7   city               10000 non-null  object 
 8   membership         10000 non-null  object 
 9   restaurant_name_y  10000 non-null  object 
 10  cuisine            10000 non-null  object 
 11  rating             10000 non-null  float64
dtypes: float64(2), int64(3), object(7)
memory usage: 937.6+ KB


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [58]:
final_df["order_date"] = pd.to_datetime(
    final_df["order_date"],
    errors="coerce"
)


In [19]:
gold_members_df = final_df[final_df['membership'] == 'Gold']
city_revenue = gold_members_df.groupby('city')['total_amount'].sum().reset_index()
highest_revenue_city = city_revenue.loc[city_revenue['total_amount'].idxmax()]
display(highest_revenue_city)

,1
city,Chennai
total_amount,1080909.79


In [20]:
cuisine_avg_order_value = final_df.groupby('cuisine')['total_amount'].mean().reset_index()
highest_avg_cuisine = cuisine_avg_order_value.loc[cuisine_avg_order_value['total_amount'].idxmax()]
display(highest_avg_cuisine)

,3
cuisine,Mexican
total_amount,808.021344


In [21]:
user_total_orders = final_df.groupby('user_id')['total_amount'].sum().reset_index()
high_value_users = user_total_orders[user_total_orders['total_amount'] > 1000]
num_distinct_users = high_value_users['user_id'].nunique()

print(f"Number of distinct users who placed orders worth more than ₹1000 in total: {num_distinct_users}")

Number of distinct users who placed orders worth more than ₹1000 in total: 2544


In [22]:
rating_bins = [0, 1, 2, 3, 4, 5]
rating_labels = ['0-1', '1-2', '2-3', '3-4', '4-5']
final_df['rating_range'] = pd.cut(final_df['rating'], bins=rating_bins, labels=rating_labels, right=False)

revenue_by_rating_range = final_df.groupby('rating_range')['total_amount'].sum().reset_index()
highest_revenue_rating_range = revenue_by_rating_range.loc[revenue_by_rating_range['total_amount'].idxmax()]
display(highest_revenue_rating_range)

/tmp/ipython-input-3569189850.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  revenue_by_rating_range = final_df.groupby('rating_range')['total_amount'].sum().reset_index()


,4
rating_range,4-5
total_amount,4315931.96


In [23]:
city_avg_order_value_gold = gold_members_df.groupby('city')['total_amount'].mean().reset_index()
highest_avg_city_gold = city_avg_order_value_gold.loc[city_avg_order_value_gold['total_amount'].idxmax()]
display(highest_avg_city_gold)

,1
city,Chennai
total_amount,808.45908


In [24]:
restaurant_counts = final_df.groupby('cuisine')['restaurant_id'].nunique().reset_index()
restaurant_counts.rename(columns={'restaurant_id': 'distinct_restaurant_count'}, inplace=True)

cumulative_revenue = final_df.groupby('cuisine')['total_amount'].sum().reset_index()
cumulative_revenue.rename(columns={'total_amount': 'total_revenue'}, inplace=True)

cuisine_summary = pd.merge(restaurant_counts, cumulative_revenue, on='cuisine')
cuisine_summary_sorted = cuisine_summary.sort_values(by=['distinct_restaurant_count', 'total_revenue'], ascending=[True, False])

display(cuisine_summary_sorted)

,cuisine,distinct_restaurant_count,total_revenue
0,Chinese,120,1930504.65
2,Italian,126,2024203.80
1,Indian,126,1971412.58
3,Mexican,128,2085503.09


In [25]:
total_orders = len(final_df)
gold_member_orders = len(gold_members_df)

percentage_gold_members = round((gold_member_orders / total_orders) * 100)

print(f"Percentage of total orders placed by Gold members: {percentage_gold_members}%")

Percentage of total orders placed by Gold members: 50%


In [26]:
restaurant_metrics = final_df.groupby(['restaurant_id', 'restaurant_name_x']).agg(
    total_orders=('order_id', 'count'),
    avg_order_value=('total_amount', 'mean')
).reset_index()

filtered_restaurants = restaurant_metrics[restaurant_metrics['total_orders'] < 20]
highest_avg_restaurant = filtered_restaurants.sort_values(by='avg_order_value', ascending=False).iloc[0]

display(highest_avg_restaurant)

,293
restaurant_id,294
restaurant_name_x,Hotel Dhaba Multicuisine
total_orders,13
avg_order_value,1040.222308


In [28]:
cuisine_city_revenue = final_df.groupby(['cuisine', 'city'])['total_amount'].sum().reset_index()
cuisine_city_revenue.head()

,cuisine,city,total_amount
0,Chinese,Bangalore,525046.13
1,Chinese,Chennai,491710.42
2,Chinese,Hyderabad,470061.40
3,Chinese,Pune,443686.70
4,Indian,Bangalore,543014.04


In [29]:
highest_revenue_combination = cuisine_city_revenue.loc[cuisine_city_revenue['total_amount'].idxmax()]
highest_revenue_combination

,12
cuisine,Mexican
city,Bangalore
total_amount,571004.61


In [32]:
final_df['order_date'] = pd.to_datetime(final_df['order_date'], format='%d-%m-%Y')
final_df['order_quarter'] = final_df['order_date'].dt.quarter
final_df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating,rating_range,order_quarter
0,1,2508,450,2023-02-18,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2,3-4,1
1,2,2693,309,2023-01-18,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5,4-5,1
2,3,2084,107,2023-07-15,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0,4-5,3
3,4,319,224,2023-10-04,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8,4-5,4
4,5,1064,293,2023-12-25,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0,3-4,4


In [38]:
final_df[["membership", "cuisine", "total_amount"]].head()
combo_revenue = (
    final_df
    .groupby(["membership", "cuisine"])["total_amount"]
    .sum()
    .reset_index()
)
combo_revenue_sorted = combo_revenue.sort_values(
    by="total_amount",
    ascending=False
)

combo_revenue_sorted
top_combo = combo_revenue_sorted.iloc[0]
top_combo
final_df.groupby(
    ["membership", "cuisine"]
)["total_amount"].sum().idxmax()



('Regular', 'Mexican')

In [61]:


final_df["order_date"] = pd.to_datetime(final_df["order_date"], errors="coerce")

highest_quarter = final_df.groupby(
    final_df["order_date"].dt.quarter
)["total_amount"].sum().idxmax()

highest_quarter
quarter_map = {
    1: "Q1 (Jan–Mar)",
    2: "Q2 (Apr–Jun)",
    3: "Q3 (Jul–Sep)",
    4: "Q4 (Oct–Dec)"
}

quarter_map[highest_quarter]



'Q3 (Jul–Sep)'

In [46]:
print(f"Total orders placed by Gold members: {gold_member_orders}")

Total orders placed by Gold members: 4987


In [47]:
hyderabad_revenue = final_df[final_df['city'] == 'Hyderabad']['total_amount'].sum()
rounded_hyderabad_revenue = round(hyderabad_revenue)
print(f"Total revenue from Hyderabad: ₹{rounded_hyderabad_revenue}")

Total revenue from Hyderabad: ₹1889367


In [48]:
num_distinct_users_total = final_df['user_id'].nunique()
print(f"Number of distinct users who placed at least one order: {num_distinct_users_total}")

Number of distinct users who placed at least one order: 2883


In [49]:
avg_order_value_gold = gold_members_df['total_amount'].mean()
rounded_avg_order_value_gold = round(avg_order_value_gold, 2)
print(f"Average order value for Gold members: ₹{rounded_avg_order_value_gold}")

Average order value for Gold members: ₹797.15


In [50]:
high_rating_restaurants_orders = final_df[final_df['rating'] >= 4.5]
num_high_rating_orders = len(high_rating_restaurants_orders)
print(f"Number of orders placed for restaurants with a rating >= 4.5: {num_high_rating_orders}")

Number of orders placed for restaurants with a rating >= 4.5: 3374


In [51]:
gold_city_revenue = gold_members_df.groupby('city')['total_amount'].sum().reset_index()
top_revenue_city_gold = gold_city_revenue.loc[gold_city_revenue['total_amount'].idxmax()]['city']

orders_in_top_gold_city = gold_members_df[gold_members_df['city'] == top_revenue_city_gold]
num_orders_top_gold_city = len(orders_in_top_gold_city)

print(f"The top revenue city among Gold members is {top_revenue_city_gold}.")
print(f"Number of orders placed in {top_revenue_city_gold} by Gold members: {num_orders_top_gold_city}")

The top revenue city among Gold members is Chennai.
Number of orders placed in Chennai by Gold members: 1337


In [57]:
print(f"Total number of rows in the final merged dataset: {final_df.shape[0]}")

Total number of rows in the final merged dataset: 10000
